A first attempt at classifying sequences. Arguably, since we're normalising away the duration of sequences to all having 128 samples, this may actually now be better characterised by a simpler CNN? 

In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
!pwd

/content


In [86]:
!ls drive/MyDrive/Colab\ Notebooks/sequence_data/

d1p01M	d1p10F	d1p19F	d1p28F	d1p37M	d1p46M	d1p55F	d2p04F	d2p13F	d2p22M
d1p02M	d1p11F	d1p20F	d1p29F	d1p38M	d1p47M	d1p56F	d2p05F	d2p14F	d2p23F
d1p03M	d1p12F	d1p21F	d1p30F	d1p39M	d1p48M	d1p57F	d2p06F	d2p15F	d2p24F
d1p04M	d1p13F	d1p22F	d1p31F	d1p40M	d1p49F	d1p58F	d2p07F	d2p16F	d2p25F
d1p05M	d1p14F	d1p23F	d1p32F	d1p41M	d1p50F	d1p59F	d2p08F	d2p17F	d2p26F
d1p06M	d1p15F	d1p24F	d1p33F	d1p42M	d1p51F	d1p60F	d2p09F	d2p18M	d2p27F
d1p07M	d1p16F	d1p25F	d1p34F	d1p43M	d1p52F	d2p01F	d2p10F	d2p19M
d1p08F	d1p17F	d1p26F	d1p35F	d1p44M	d1p53F	d2p02F	d2p11F	d2p20M
d1p09F	d1p18F	d1p27F	d1p36M	d1p45M	d1p54F	d2p03F	d2p12F	d2p21M


In [87]:
%run drive/MyDrive/Colab\ Notebooks/scripts/lstm_dataprep.py

Index(['participant', 'activity', 'activity_label', 'activity_sequence',
       't_norm', 'front', 'lateral', 'vertical', 'rssi', 'phase', 'frequency'],
      dtype='object')


In [88]:
norm_df.head()

,participant,activity,activity_label,activity_sequence,t_norm,front,lateral,vertical,rssi,phase,frequency
0,d1p01M,1,bed,1,0.000000,0.272030,-0.082102,1.00820,-63.500000,2.425200,924.250000
1,d1p01M,1,bed,1,0.007874,0.272030,-0.082102,1.00820,-63.287402,3.408128,923.187008
2,d1p01M,1,bed,1,0.015748,0.272030,-0.082102,1.00820,-63.074803,4.391055,922.124016
3,d1p01M,1,bed,1,0.023622,0.320501,-0.063247,0.98289,-63.137795,4.266798,922.301181
4,d1p01M,1,bed,1,0.031496,0.395284,-0.034156,0.94384,-63.350394,3.541497,923.151575


##let's see if we can overfit on a mini dataset

prepare mini train & test datasets

In [89]:
Y = []
X = []

for name, group in norm_df.groupby(['participant', 'activity_sequence']):
  Y.append(group['activity'].unique())
  X.append(group[['front', 'lateral', 'vertical', 'rssi', 'phase', 'frequency']].to_numpy())

In [90]:
mini_X = X[:12]
mini_Y = Y[:12]

In [91]:
mini_Y

[array([1]),
 array([3]),
 array([1]),
 array([4]),
 array([2]),
 array([4]),
 array([1]),
 array([3]),
 array([1]),
 array([4]),
 array([1]),
 array([3])]

In [92]:
# x = torch.tensor(x,dtype=torch.float32)

make mini train & test

In [93]:
y_test = mini_Y[4:8]
del mini_Y[4:8]
y_train = mini_Y

In [94]:
x_test = mini_X[4:8]
del mini_X[4:8]
x_train = mini_X

In [95]:
x_test = torch.tensor(np.array(x_test),dtype=torch.float64)
x_train = torch.tensor(np.array(x_train),dtype=torch.float64)
y_test = torch.tensor(np.array(y_test),dtype=torch.float64)
y_train = torch.tensor(np.array(y_train),dtype=torch.float64)

In [96]:
y_test

tensor([[2.],
        [4.],
        [1.],
        [3.]], dtype=torch.float64)

In [97]:
# one-hot encode y test & train
# careful we're not adjusting for the fact that one-hot starts and 0 and introduces redundant 
# initial 0 class - TODO either adjust for in later code or fix here
y_test = torch.nn.functional.one_hot(y_test.to(torch.int64))
y_train = torch.nn.functional.one_hot(y_train.to(torch.int64))

In [98]:
y_test

tensor([[[0, 0, 1, 0, 0]],

        [[0, 0, 0, 0, 1]],

        [[0, 1, 0, 0, 0]],

        [[0, 0, 0, 1, 0]]])

create train & test datasets

In [99]:
train_data = TensorDataset(x_train, y_train)
test_data = TensorDataset(x_test, y_test)

Define static variables

In [111]:
batch_size = 2   # since data is mini!
input_size = 6  # no of variables - may be wrong!
hidden_size = 32 # number of hidden neurons - should ensure we overfit
layer_size = 2   # number of layers
output_size = 10 # number of classes 

training and evaluation functions

In [112]:
def get_accuracy(out, actual_labels, batchSize):
    '''Saves the Accuracy of the batch.
    Takes in the log probabilities, actual label and the batchSize (to average the score).'''
    predictions = out.max(dim=1)[1]
    # ajust for the fact one-hot encoding starts at 0
    correct = (predictions == actual_labels).sum().item() + 1
    accuracy = correct/batch_size
    
    return accuracy

In [113]:
def train_network(model, train_data, test_data, batchSize=64, num_epochs=1, learning_rate=0.001):
    
    '''Trains the model and computes the average accuracy for train and test data.'''
    
    print('Get data ready...')
    # Create dataloader for training dataset - so we can train on multiple batches
    # Shuffle after every epoch
    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batchSize, shuffle=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, drop_last=True)
    
    # Create criterion and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    
    print('Training started...')
    # Train the data multiple times
    for epoch in range(num_epochs):
        
        # Save Train and Test Loss
        train_loss = 0
        train_acc = 0
        
        # Set model in training mode:
        model.train()
        
        for k, (seq, labels) in enumerate(train_loader):
            
            # Create log probabilities
            out = model(seq)
            # Clears the gradients from previous iteration
            optimizer.zero_grad()
            # Computes loss: how far is the prediction from the actual?
            loss = criterion(out, labels)
            # Computes gradients for neurons
            loss.backward()
            # Updates the weights
            optimizer.step()
            
            # Save Loss & Accuracy after each iteration
            train_loss += loss.item()
            train_acc += get_accuracy(out, labels, batchSize)
            
        
        # Print Average Train Loss & Accuracy after each epoch
        print('TRAIN | Epoch: {}/{} | Loss: {:.2f} | Accuracy: {:.2f}'.format(epoch+1, num_epochs, train_loss/k, train_acc/k))
            
            
    print('Testing Started...')
    # Save Test Accuracy
    test_acc = 0
    # Evaluation mode
    model.eval()
    
    for k, (seq, labels) in enumerate(test_loader):
        
        # Create logit predictions
        out = model(images)
        # Add Accuracy of this batch
        test_acc += get_accuracy(out, labels, batchSize)
        
    # Print Final Test Accuracy
    print('TEST | Average Accuracy per {} Loaders: {:.5f}'.format(k, test_acc/k) )

Adapted from a tutorial on image classification - doesn't work yet. I suspect un-stacking the 6 features into a single vector may work. Still need to work through the dimensions to ensure they're correct. To be continued! 

In [114]:


class LSTM_SEQ(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, bidirectional=True):
        super(LSTM_SEQ, self).__init__()
        
        self.input_size, self.hidden_size, self.layer_size, self.output_size = input_size, hidden_size, layer_size, output_size
        self.bidirectional = bidirectional
        
        # Step1: the LSTM model
        self.lstm = nn.LSTM(input_size, hidden_size, layer_size, batch_first=True, bidirectional=bidirectional)
        
        # Step2: the FNN
        if bidirectional: # we'll have 2 more layers
            self.layer = nn.Linear(hidden_size*2, output_size)
        else:
            self.layer = nn.Linear(hidden_size, output_size)
            
            
    def forward(self, sequences, prints=True):
        if prints: print('sequences shape:', sequences.shape)
        
        # Set initial states
        if self.bidirectional:
            # Hidden state:
            hidden_state = torch.zeros(self.layer_size*2, sequences.size(0), self.hidden_size)
            # Cell state:
            cell_state = torch.zeros(self.layer_size*2, sequences.size(0), self.hidden_size)
        else:
            # Hidden state:
            hidden_state = torch.zeros(self.layer_size, sequences.size(0), self.hidden_size)
            # Cell state:
            cell_state = torch.zeros(self.layer_size, sequences.size(0), self.hidden_size)
        if prints: print('hidden_state t0 shape:', hidden_state.shape, '\n' +
                         'cell_state t0 shape:', cell_state.shape)
        
        # LSTM:
        output, (last_hidden_state, last_cell_state) = self.lstm(sequences, (hidden_state, cell_state))
        if prints: print('LSTM: output shape:', output.shape, '\n' +
                         'LSTM: last_hidden_state shape:', last_hidden_state.shape, '\n' +
                         'LSTM: last_cell_state shape:', last_cell_state.shape)
        # Reshape
        output = output[:, -1, :]
        if prints: print('output reshape:', output.shape)
        
        # FNN:
        output = self.layer(output)
        if prints: print('FNN: Final output shape:', output.shape)
        
        return output



In [115]:
lstm_example = LSTM_SEQ(input_size, hidden_size, layer_size, output_size)
print('lstm_example:', lstm_example, '\n')


lstm_example: LSTM_SEQ(
  (lstm): LSTM(6, 32, num_layers=2, batch_first=True, bidirectional=True)
  (layer): Linear(in_features=64, out_features=10, bias=True)
) 



In [116]:

# Instantiate the model
lstm_rnn = LSTM_SEQ(input_size, hidden_size, layer_size, output_size)

# ==== TRAIN ====
train_network(lstm_rnn, train_data, test_data, batchSize=2, num_epochs=10)

#train_network(model, train_data, test_data, batchSize=64, num_epochs=1, learning_rate=0.001):



Get data ready...
Training started...
sequences shape: torch.Size([2, 128, 6])
hidden_state t0 shape: torch.Size([4, 2, 32]) 
cell_state t0 shape: torch.Size([4, 2, 32])


RuntimeError: ignored